In [1]:
#@title Função
def preprocessar_forms(idade, imc, atv_fisica, estrogenio, qtd_estrogenio, progesterona, qtd_progesterona, ki67, qtd_ki67, tnmt, tnmn, tnmm, estadio, her2, subtipo):
  import pandas as pd
  import numpy as np
  import math
  from sklearn.preprocessing import MinMaxScaler
  from sklearn.ensemble import RandomForestClassifier
  import joblib
  import io

  estrogenio = estrogenio.lower()
  progesterona = progesterona.lower()
  ki67 = ki67.lower()
  estadio = estadio.lower()

  df = pd.DataFrame([[idade, imc, atv_fisica, estrogenio, qtd_estrogenio, progesterona, qtd_progesterona, ki67, qtd_ki67, tnmt, tnmn, tnmm, subtipo, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],columns=['idade_primeiro_diagnostico', 'imc', 'atividade_fisica', 'receptor_de_estrogenio', 'estrogenio_qtd', 'receptor_de_progesterona', 'progesterona_qtd', 'ki67_14', 'ki67_qtd', 'classificacao_tnm_clinico_t', 'classificacao_tnm_clinico_n', 'classificacao_tnm_clinico_m', 'subtipo_tumoral', 'est_c_iiia', 'est_c_iib', 'est_c_iia', 'est_c_i', 'est_c_iiib', 'est_c_iiic', 'est_c_0', 'est_c_iv', 'est_c_ia', 'est_c_ib', "her2_+++_positivo", 'her2_+_negativo', 'her2_0_negativo', 'her2_indeterminado', 'her2_++_duvidoso'])

  for column in df.columns:
    if column[6:] == estadio:
      df[column] = 1

  if her2 == ' +++ (positivo)':
    df["her2_+++_positivo"] = 1
  if her2 == '0 (negativo)':
    df['her2_0_negativo'] = 1
  if her2 == ' + (negativo)':
    df['her2_+_negativo'] = 1
  if her2 == ' ++ (duvidoso)':
    df['her2_++_duvidoso'] = 1
  if her2 == 'indeterminado':
    df['her2_indeterminado'] = 1

  # Agrupando classificação T
  df['classificacao_tnm_clinico_t'] = df['classificacao_tnm_clinico_t'].replace({
      '1MIC': '1', # 1
      '1A': '1',
      '1B': '1',
      '1C': '1',
      '2A': '2', # 2
      '2B': '2',
      '3A': '3', # 3
      '3B': '3',
      '4C': '4', # 4
      '4A': '4',
      '4B': '4',
      '4D': '4',
      'CDIS': 'IS', # IS
      'CLIS': 'IS'
  })

  # Transformando A em 0
  df = df[~df['classificacao_tnm_clinico_t'].isin(['A', '0', 'Y: Na', 'X - nao foi possivel determinar'])]

  # Agrupando classificação N
  df['classificacao_tnm_clinico_n'] = df['classificacao_tnm_clinico_n'].replace({
      '1MIC': '1', # 1
      '1A': '1',
      '1B': '1',
      '1C': '1',
      '2A': '2', # 2
      '2B': '2',
      '3A': '3', # 3
      '3B': '3',
      '3C': '3',
      '4C': '4', # 4
      '4A': '4',
      '4B': '4',
      '4D': '4'
  })

  # Agrupando classificação M
  df['classificacao_tnm_clinico_m'] = df['classificacao_tnm_clinico_m'].replace({
      '1MIC': '1', # 1
      '1A': '1',
      '1B': '1',
      '1C': '1',
      '2A': '2', # 2
      '2B': '2',
      '3A': '3', # 3
      '3B': '3',
      '3C': '3',
      '4C': '4', # 4
      '4A': '4',
      '4B': '4',
      '4D': '4'
  })

  cols = ['classificacao_tnm_clinico_t', 'classificacao_tnm_clinico_n',
       'classificacao_tnm_clinico_m', 'subtipo_tumoral',
       'receptor_de_estrogenio', 'receptor_de_progesterona', 'ki67_14',
       'atividade_fisica', 'est_c_0', 'est_c_i',
       'est_c_ia', 'est_c_ib', 'est_c_iia', 'est_c_iib', 'est_c_iiia',
       'est_c_iiib', 'est_c_iiic', 'est_c_iv', 'her2_+_negativo',
       'her2_++_duvidoso', 'her2_+++_positivo', 'her2_0_negativo',
       'her2_indeterminado', 'imc', 'estrogenio_qtd', 'progesterona_qtd',
       'ki67_qtd', 'idade_primeiro_diagnostico']
  df = df[cols]

  !gdown '1aOhfiEo_sAseqTz5UKZuUY5iKH4Wyszv'
  !gdown '1bIRe2Xkzxdtk5_5BsKXE5cfl84fT0zBO'
  scaler = joblib.load('/content/minMax.scaler')
  rf_classifier = joblib.load('/content/modelo_rf_classifier.model')

  def replace_arr(base_str, values_to_replace, replace_str):
    for value in values_to_replace:
      base_str = base_str.replace(value, replace_str)
    return base_str

  def normalize_columns_name(df):
    columns = []
    for column in df.columns:
      column = column.lower().strip()
      column = replace_arr(column, [
          '?', '(', ')', '!', 'º', '[', ']', '%', '>', '<', ':', ',', ';', '.', '#', '/' 
      ], '')
      column = replace_arr(column, [
          'á', 'à', 'â', 'ã',
      ], 'a')
      column = replace_arr(column, [
          'é', 'è', 'ê',
      ], 'e')
      column = replace_arr(column, [
          'í', 'ì',
      ], 'i')
      column = replace_arr(column, [
          'ó', 'ò', 'ô', 'õ'
      ], 'o')
      column = replace_arr(column, [
          'ú', 'ù', 'û'
      ], 'u')
      column = replace_arr(column, [
          '-', ' ', '=', ',', '__', '__'
      ], '_')
      column = replace_arr(column, [
          'ç'
      ], 'c')

      if column[-1] == '_':
        column = column[:-1]

      if column[0] == '_':
        column = column[1:]
      
      columns.append(column)

    df.columns = columns
    return df

  # cria uma lista de colunas numéricas a serem selecionadas
  colunas_numericas = ['imc', 'estrogenio_qtd', 'progesterona_qtd', 'ki67_qtd', 'idade_primeiro_diagnostico']
  # cria um novo DataFrame com apenas as colunas numéricas especificadas
  df_numericas = df[colunas_numericas]
  # cria um novo DataFrame com todas as colunas exceto as colunas numéricas e a coluna 'record_id'
  df_categoricas = df.drop(columns=[*colunas_numericas])

  # transforma os valores das colunas categóricas em numéricos
  df_categoricas['classificacao_tnm_clinico_t'] = df_categoricas['classificacao_tnm_clinico_t'].replace({
      'IS': 0
  }).astype(int)

  df_categoricas['receptor_de_estrogenio'] = df_categoricas['receptor_de_estrogenio'].replace({
      'negativo': 0,
      'positivo': 1
  }).astype(int)

  df_categoricas['receptor_de_progesterona'] = df_categoricas['receptor_de_progesterona'].replace({
      'negativo': 0,
      'positivo': 1
  }).astype(int)

  df_categoricas['ki67_14'] = df_categoricas['ki67_14'].replace({
      'negativo': 0,
      'positivo': 1
  }).astype(int)

  df_categoricas['classificacao_tnm_clinico_n'] = df_categoricas['classificacao_tnm_clinico_n'].astype(int)

  df_categoricas['classificacao_tnm_clinico_m'] = df_categoricas['classificacao_tnm_clinico_m'].astype(int)

  df_categoricas['subtipo_tumoral'] = df_categoricas['subtipo_tumoral'].astype(int)


  # preenche com zero os valores nulos da coluna `atividade_fisica`
  df_categoricas.loc[df_categoricas['atividade_fisica'].isna(), 'atividade_fisica'] = 0
  # transforma os valores da coluna `atividade_fisica`
  df_categoricas['atividade_fisica'] = df_categoricas['atividade_fisica'].replace({
      'Não pratica': -1,
      'Pratica esporadicamente': 1,
      'Pratica regularmente': 2,
      'Pratica frequentemente': 3
  }).astype(int)

  # Merge categóricas e numéricas
  df = pd.concat([df_categoricas, df_numericas], axis=1)

  # definindo x e y para separar em treino e teste
  df_x = df

  df_x[colunas_numericas] = scaler.transform(df_x[colunas_numericas])

  # Realiza as predições
  predicoes = rf_classifier.predict(df_x)
  # Troca as predições de 0 e 1 para strings
  predicoes_em_string = np.where(predicoes == 1, 'Neoadjuvante', 'Adjuvante')
  return str(predicoes_em_string[0])

In [2]:
#@title Gradio
%%capture
!pip install gradio
import gradio as gr

idade = gr.Slider(0, 100, value=4, label="Idade diagnostico", info="Selecione a quantidade")
imc = gr.Slider(15, 40, value=4, label="Valor do IMC", info="Selecione a quantidade")
atv_fisica = gr.Dropdown(['Pratica esporadicamente', 'Não pratica', 'Pratica regularmente', 'Pratica frequentemente'], label="Atividade Física")
estrogenio = gr.Dropdown(['Positivo', 'Negativo'], label="Receptor de estrogênio")
qtd_estrogenio = gr.Slider(0, 200, value=4, label="Quantidade de estrogenio?", info="Selecione a quantidade")
progesterona = gr.Dropdown(['Positivo', 'Negativo'], label="Receptor de progesterona")
qtd_progesterona = gr.Slider(0, 200, value=4, label="Quantidade de progesterona?", info="Selecione a quantidade")
ki67 = gr.Dropdown(['Positivo', 'Negativo'], label="KI67-14")
qtd_ki67 = gr.Slider(0, 200, value=4, label="Quantidade de Ki67?", info="Selecione a quantidade")
tnmt = gr.Dropdown(["1MIC", "1A", "1B", "1C", "2A", "2B", "3A", "3B", "4A", "4B", "4C", "4D", "CDIS", "CLIS"], label="Qual a classificação do TNM-T")
tnmn = gr.Dropdown(["1MIC", "1A", "1B", "1C", "2A", "2B", "3A", "3B", "3C", "4A", "4B", "4C", "4D"], label="Qual a classificação do TNM-N")
tnmm = gr.Dropdown(["1MIC", "1A", "1B", "1C", "2A", "2B", "3A", "3B", "3C", "4A", "4B", "4C", "4D"], label="Qual a classificação do TNM-M")
estadio = gr.Dropdown(["0", "IA", "IIA", "IIIA", "IVA" "IB", "IIB", "IIIB", "IVB", "IC", "IIC", "IIIC", "IVC"], label="Qual o tipo do estadio clinico?")
her2 = gr.Dropdown([' +++ (positivo)', '0 (negativo)', ' + (negativo)', ' ++ (duvidoso)', 'indeterminado'], label="HER2")
subtipo = gr.Dropdown(["1","2","3","4"], label="Qual o subtipo tumoral?")


interface = gr.Interface(
    fn=preprocessar_forms,
    inputs=[idade, imc, atv_fisica, estrogenio, qtd_estrogenio, progesterona, qtd_progesterona, ki67, qtd_ki67, tnmt, tnmn, tnmm, estadio, her2, subtipo],
    outputs="text",
    title="Modelo para predição do melhor tratamento do câncer",
    description="Preencha os campos abaixo e clique em 'Enviar' para obter o resultado."
)

In [3]:
interface.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>